In [1]:
!pip install accelerate peft bitsandbytes transformers trl tqdm pandas torch transformers

     |████████████████████████████████| 227 kB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 56 kB 12.4 MB/s eta 0:00:01
     |████████████████████████████████| 105.0 MB 17.1 MB/s eta 0:00:01    |████████████▌                   | 41.0 MB 12.6 MB/s eta 0:00:06
     |████████████████████████████████| 7.2 MB 19.5 MB/s eta 0:00:01
     |████████████████████████████████| 226 kB 18.3 MB/s eta 0:00:01
     |████████████████████████████████| 11.0 MB 9.4 MB/s eta 0:00:01
     |████████████████████████████████| 189 kB 17.5 MB/s eta 0:00:01
     |████████████████████████████████| 250 kB 46.5 MB/s eta 0:00:01
     |████████████████████████████████| 135.3 MB 243 kB/s  eta 0:00:01
     |████████████████████████████████| 4.0 MB 17.6 MB/s eta 0:00:01
     |████████████████████████████████| 416 kB 20.1 MB/s eta 0:00:01
     |████████████████████████████████| 268 kB 52.4 MB/s eta 0:00:01
     |████████████████████████████████| 512 kB 22.0 MB/s eta 0:00:01
     |███████████████████████████

In [2]:
import pandas as pd
import torch
import pandas as pd
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
from util import format_prompt_test, generate, extract_text_after_tag
from tqdm import tqdm

# Load dataset
df = pd.read_csv('eval_data_6_30.csv')
df['input'] = df.apply(lambda row: format_prompt_test(row['essay'], row['prompt']), axis=1)

In [4]:
df.head()

,essay,prompt,input
0,"Dear Principal,\n\nI have heard you are having...",Some of your friends perform community service...,### INSTRUCTIONS: You are my English teacher. ...
1,Asking for more than one person for advice can...,"When people ask for advice, they sometimes tal...",### INSTRUCTIONS: You are my English teacher. ...
2,"Dear Principal,\n\nConsidering a policy change...",Your principal is considering changing school ...,### INSTRUCTIONS: You are my English teacher. ...
3,"2/1/11\n\nDear Mr. Principal,\n\nI believe tha...",Some of your friends perform community service...,### INSTRUCTIONS: You are my English teacher. ...
4,The New school policy\n\nI do not like the new...,Your principal is considering changing school ...,### INSTRUCTIONS: You are my English teacher. ...


In [3]:
df.shape

(174, 3)

In [5]:
# Set up model config
base_model = "NousResearch/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
compute_dtype = getattr(torch, "float16")
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:15                                                                                   │
│                                                                                                  │
│   12 )                                                                                           │
│   13 model = AutoModelForCausalLM.from_pretrained(                                               │
│   14 │   base_model,                                                                             │
│ ❱ 15 │   device_map={"": 0}                                                                      │
│   16 )                                                                                           │
│   17 model.config.use_cache = False                                                              │
│   18 model.config.pretraining_tp = 1                                                             │
│                                                                                                  │
│ /Applications/anaconda3/envs/cs224n/lib/python3.7/site-packages/transformers/models/auto/auto_fa │
│ ctory.py:485 in from_pretrained                                                                  │
│                                                                                                  │
│   482 │   │   elif type(config) in cls._model_mapping.keys():                                    │
│   483 │   │   │   model_class = _get_model_class(config, cls._model_mapping)                     │
│   484 │   │   │   return model_class.from_pretrained(                                            │
│ ❱ 485 │   │   │   │   pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs,   │
│   486 │   │   │   )                                                                              │
│   487 │   │   raise ValueError(                                                                  │
│   488 │   │   │   f"Unrecognized configuration class {config.__class__} for this kind of AutoM   │
│                                                                                                  │
│ /Applications/anaconda3/envs/cs224n/lib/python3.7/site-packages/transformers/modeling_utils.py:2 │
│ 597 in from_pretrained                                                                           │
│                                                                                                  │
│   2594 │   │   │   │   user_agent=user_agent,                                                    │
│   2595 │   │   │   │   revision=revision,                                                        │
│   2596 │   │   │   │   subfolder=subfolder,                                                      │
│ ❱ 2597 │   │   │   │   _commit_hash=commit_hash,                                                 │
│   2598 │   │   │   )                                                                             │
│   2599 │   │                                                                                     │
│   2600 │   │   # load pt weights early so that we know which dtype to init the model under       │
│                                                                                                  │
│ /Applications/anaconda3/envs/cs224n/lib/python3.7/site-packages/transformers/utils/hub.py:970 in │
│ get_checkpoint_shard_files                                                                       │
│                                                                                                  │
│    967 │   │   │   │   user_agent=user_agent,                                                    │
│    968 │   │   │   │   revision=revision,                                                        │
│    969 │   │   │   │   subfolder=subfolder,                                                      │
│ ❱  970 │   │   │   │   _commit_hash=_commit_hash,          

In [ ]:
#try just 1
res = generate(df['input'][0], model, tokenizer)
print(res)

In [ ]:
tqdm.pandas()
df['predictions'] = df.progress_apply(lambda row: generate(row['input'], model, tokenizer), axis=1)
df['predictions'][0]

In [ ]:
df['extracted_content'] = df['predictions'].apply(extract_text_after_tag)